In [ ]:
import logging
import os
import warnings
from pathlib import Path

import hydra
import rootutils
from hydra.core.hydra_config import HydraConfig
from kaggle import KaggleApi

rootutils.setup_root(search_from="../", indicator=".project-root", pythonpath=True)

from src.utils.log_utils import get_consol_handler, get_file_handler

In [ ]:
OVERRIDES: list[str] = os.getenv("OVERRIDES", "experiment=default").split(",")

In [ ]:
if OVERRIDES is None:
    raise ValueError("OVERRIDES is not set")

with hydra.initialize(version_base=None, config_path="../configs"):
    CFG = hydra.compose(
        config_name="config.yaml",
        return_hydra_config=True,
        overrides=OVERRIDES,
    )
    # use HydraConfig for notebook to use hydra job
    HydraConfig.instance().set_config(CFG)


if CFG.debug:
    CFG.paths.output_dir = f"{CFG.paths.output_dir}_debug"

INPUT_DIR = Path(CFG.paths.input_dir)  # input directory (r/w)
OUTPUT_DIR = Path(CFG.paths.output_dir)  # experiment output directory (r/w)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_ROOT_DIR = Path(CFG.paths.data_dir) / "output"  # output directory (r/w)
COMPETITION_DATA_DIR = INPUT_DIR / CFG.meta.competition_name

# clients
KAGGLE_CLIENT = KaggleApi()
KAGGLE_CLIENT.authenticate()

# set logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [
    get_file_handler(OUTPUT_DIR / "notebook.log"),
    get_consol_handler(),
]

warnings.filterwarnings("ignore")